In [2]:
import fasttext
import numpy as np
import math
from numpy import linalg as lin
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import gensim
from gensim.models import Word2Vec 
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

In [3]:
word_array=[]
with open('./data.txt','r') as f1:
    for line in f1:
        words=word_tokenize(line)
        for i in range(len(words)):
            if(words[i] not in punctuations):
                word_array.append(words[i].lower())

print(len(word_array))

7247443


In [4]:
with open('./sravya_train.txt', 'w') as f1:
    for i in range(len(word_array)):
        f1.write(word_array[i]+'\n')

model = fasttext.skipgram('./sravya_train.txt', 'model')
# model = gensim.models.Word2Vec(word_array, min_count = 1,  
#                               size = 100, window = 5) 

In [5]:
def sent2vec(sent):
    words=word_tokenize(sent)
    final_words=[]
    for i in range(len(words)):
        if(words[i] not in punctuations):
            final_words.append(words[i].lower())
    ans=[]
    for i in range(100):
        ans.append(0)
    for i in range(len(final_words)):
        val=model[final_words[i]]
        for j in range(len(ans)):
            ans[j]+=val[j]
    n=len(final_words)
    if(n!=0):
        for i in range(len(ans)):
            ans[i]=ans[i]/n
#     print(ans)
    return ans

In [6]:
def cosine_similarity(vec1,vec2):
    mag1=0
    mag2=0
    num=0
    for i in range(100):
        mag1+=(vec1[i]*vec1[i])
        mag2+=(vec2[i]*vec2[i])
        num+=(vec1[i]*vec2[i])
    mag=mag1*mag2
    mag=math.sqrt(mag)
    if(mag==0):
        return -10
    ans=num/mag
    return ans

In [16]:
sent1="This is a new sentence"
sent2="This is new sentence"
vec1=sent2vec(sent1)
vec2=sent2vec(sent2)
# for i in range(100):
#     print(str(vec1[i])+'\t'+str(vec2[i]))
ans=cosine_similarity(vec1,vec2)
print(ans)

0.9999933556880889


In [8]:
train_array=[]
train_ans=[]
with open('train_q1.txt','r') as f1:
    with open('train_q2.txt','r') as f2:
        with open('train-ans.txt','r') as f3:
            for line1,line2,line3 in zip(f1,f2,f3):
                sent1=sent2vec(line1)
                sent2=sent2vec(line2)
                val1=cosine_similarity(sent1,sent2)
                if(val1!=-10):
                    sravya=[val1,1.00]
                    train_array.append(sravya)
                    train_ans.append(line3.strip('\n'))

for i in range(len(train_array)):
    ans=1+(train_array[i][0]**2)
    ans=math.sqrt(ans)
    train_array[i][0]/=ans
    train_array[i][1]/=ans

In [9]:
w_matrix=np.zeros((2,2))
rows=len(train_array)

for it in range(8):
    delta_j=np.zeros((2,2))
    for j in range(2):
        for k in range(rows):
            prob=(math.e)**(w_matrix[j][0]*train_array[k][0] + w_matrix[j][1]*train_array[k][1])           
            total=0
            for p in range(2):
                total+=((math.e)**(w_matrix[p][0]*train_array[k][0] + w_matrix[p][1]*train_array[k][1]))
            prob = prob/total
            if((j==0 and train_ans[k]=='0') or (j==1 and train_ans[k]=='1')):
                prob=1-prob
            else:
                prob=-prob
            delta_j[j]-=np.multiply(train_array[k],prob)
        w_matrix[j]-=np.multiply(delta_j[j],0.1)
    val=w_matrix[0][0]**2 + w_matrix[0][1]**2
    val=math.sqrt(val)
    w_matrix[0][0]/=val
    w_matrix[0][1]/=val
    val=w_matrix[1][0]**2 + w_matrix[1][1]**2
    val=math.sqrt(val)
    w_matrix[1][0]/=val
    w_matrix[1][1]/=val   

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [10]:
print(w_matrix)

[[0.70708098 0.70713258]
 [0.70707405 0.70713951]]


In [14]:
acc=0
cnt1=0
cnt2=0
cnt=0
acc1=0
acc2=0
with open('q1.txt','r') as f1:
    with open('q2.txt','r') as f2:
        with open('answers.txt','r') as f3:
            for line1,line2,line3 in zip(f1,f2,f3):
                cnt+=1
                if(line1!='\n' and line2!='\n' and line3!='\n'):
                    sent1=sent2vec(line1)
                    sent2=sent2vec(line2)
                    val=0
                    val1=cosine_similarity(sent1,sent2)
                    val11=(math.e)**(val1*w_matrix[0][0]+w_matrix[0][1])
                    val12=(math.e)**(val1*w_matrix[1][0]+w_matrix[1][1])
                    if(val11>=val12):
                        val=0
                    else:
                        val=1
                    if(line3.strip('\n')=='1' and val==1):
                        acc2+=1
                    if(line3.strip('\n')=='0' and val==0):
                        acc1+=1
                    if(line3.strip('\n')=='1'):
                        cnt2+=1
                    if(line3.strip('\n')=='0'):
                        cnt1+=1
         
acc=acc1+acc2

In [15]:
print("Correct answers   : " +str(acc))
print("Total test cases  : "+ str(cnt))
print("Accuracy          : "+str(acc*100/cnt))

Correct answers   : 48475
Total test cases  : 80872
Accuracy          : 59.94039964388169
